### RatNDay4 Openfield recording info

In [1]:
import numpy as np
from subjects import of

sess = of.ratNday4[0]
print(sess.recinfo)

filename: /data/Clustering/sessions/RatN/Day4/RatN_Day4_2019-10-15_11-30-06.xml 
# channels: 134
sampling rate: 30000
lfp Srate (downsampled): 1250



### Set probe configuration
This animal had 8 shank probe from diagnostic biochips

In [ ]:
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=20,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)

prbgrp.filename = sess.filePrefix.with_suffix(".probegroup.npy")
prbgrp.save()
plot_probe(prbgrp)



### Experimental Paradigm 

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0, 7423, 11497,10186],
        "stop": [7407, 11483, 25987,11483],
        "label": ["pre", "maze", "post",'sprinkle'],
    }
)

a = Epoch(epochs=epochs)
a.filename = sess.filePrefix.with_suffix(".paradigm.npy")
a.save()


### Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

### Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([1])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
plotting.plot_artifact_epochs(artifact_epochs, signal)


## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

### Neurons
- import spiketrains from Phy
- estimate neuron types such pyramidal, interneuron etc. 

## Importing spiketrains from Phy

In [2]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path(
    "/data/Clustering/sessions/RatN/Day4/spykcirc/RatN_Day4_2019-10-15_11-30-06.GUI"
)

# chan_grps = sess.recinfo.channel_groups
# phy_data = PhyIO(cluster_path)
# spiketrains =phy_data.spiketrains
# peak_chans = phy_data.peak_channels
# waveforms = phy_data.peak_waveforms
# shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

# neurons = Neurons(
#     np.array(spiketrains, dtype=object),
#     t_stop=sess.eegfile.duration,
#     sampling_rate=phy_data.sampling_rate,
#     peak_channels=peak_chans,
#     waveforms=np.array(waveforms,dtype='object'),
#     shank_ids=np.array(shank_id).astype(int),
#     metadata={'cluster_path':cluster_path}
# )

chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains = phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id < 4] = "pyr"
neuron_type[neuron_type_id == 6] = "mua"
neuron_type[neuron_type_id == 8] = "inter"


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms, dtype="object"),
    shank_ids=np.array(shank_id).astype(int),
    neuron_type=neuron_type,
)

neurons.save(sess.filePrefix.with_suffix(".neurons"))


/data/Clustering/sessions/RatN/Day4/RatN_Day4_2019-10-15_11-30-06.neurons saved


## Estimate neuron type

In [ ]:
%matplotlib widget
from neuropy.utils import neurons_util
neuron_type = neurons_util.estimate_neuron_type(sess.neurons)
sess.neurons.neuron_type = neuron_type

In [ ]:
neurons.filename = sess.filePrefix.with_suffix('.neurons')
neurons.save()

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster

plot_raster(neurons,color='jet',add_vert_jitter=True)

## BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()


### Assign position data

In [4]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / 'position'
opti_data = OptitrackIO(dirname=opti_folder,scale_factor=0.25)

/data/Clustering/sessions/RatN/Day4/position/Take 2019-10-15 11.30.03 AM.csv
/data/Clustering/sessions/RatN/Day4/position/Take 2019-10-15 01.34.34 PM.csv
/data/Clustering/sessions/RatN/Day4/position/Take 2019-10-15 02.46.47 PM.csv


In [6]:
import pandas as pd
from datetime import datetime

# ------- maze align corection ---------
t_error = [1.2,0]

# ---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(
        t_error[i], unit="sec"
    )
    print(tbegin)
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)

x, y, z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z, x, y))

position = Position(traces=traces, t_start=0, sampling_rate=opti_data.sampling_rate)
position.save(sess.filePrefix.with_suffix(".position.npy"))


2019-10-15 11:30:07.200000
2019-10-15 14:46:50
/data/Clustering/sessions/RatN/Day4/RatN_Day4_2019-10-15_11-30-06.position.npy saved


In [ ]:
import matplotlib.pyplot as plt

# plt.plot(opti_data.datetime_array,opti_data.z)
plt.plot(position.x,position.y)

In [ ]:
%matplotlib widget
from neuropy.utils import signal_process

signal = sess.eegfile.get_signal([40])

spec = signal_process.spectrogramBands(signal.traces[0])


In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt

plt.plot(spec.time,spec.theta/500)
plt.plot(sess.position.time,sess.position.y)
plt.plot(sess.position.time[1:],np.diff(sess.position.y)*100)
plt.xlim([12500,12600])
plt.ylim([-400,400])

## Linearize position

In [ ]:
from neuropy.utils import position_util

maze = sess.paradigm['maze']
maze_pos = sess.position.time_slice(maze[0],maze[1])
linear_pos = position_util.linearize_position(maze_pos)

In [ ]:
linear_pos.y

In [ ]:
%matplotlib widget
plt.plot(maze_pos.time,maze_pos.x)
plt.plot(linear_pos.time,linear_pos.x)

In [ ]:
import numpy as np

a = np.ones(5)
np.vstack((a,a,a)).shape

In [ ]:
from neuropy.core import animal

d = {'name':'hello','tag':'ser'}

an = animal.Animal(d) 
